<a href="https://colab.research.google.com/github/Bianca200303/recomendaciones/blob/main/recomendacionvideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RECOMENDACIÓN DE VIDEOS**




In [1]:
# Upload dataset

from google.colab import files
uploaded = files.upload()

Saving GBvideos.csv to GBvideos.csv


Este proyecto implementa un sistema de recomendación basado en contenido utilizando Python. El objetivo es recomendar videos de YouTube similares a un video seleccionado por el usuario, basándose en las etiquetas asociadas a cada video. A continuación, se describe el flujo del código y los componentes clave del sistema de recomendación:


In [1]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [31]:
df = pd.read_csv('GBvideos.csv')
df_sample = df.sample(n=1000)
df_sample=df_sample.reset_index(drop=True)

print(len(df_sample.index))
print(df_sample.shape)
#explorando etiquetas
print(df_sample['tags'].head)

1000
(1000, 16)
<bound method NDFrame.head of 0                                                 [none]
1      the four|"singing"|"dj khaled"|"diddy"|"meghan...
2                                                 [none]
3      words you cant say anymore|"the m word"|"now t...
4      SUPER JUNIOR|"슈퍼주니어"|"은혁"|"동해"|"예성"|"신동"|"시원"|...
                             ...                        
995    overwatch|"overwatchleague"|"competitive"|"red...
996    Eminem|"Framed"|"Aftermath"|"New"|"JV-Old"|"Sc...
997    cobra kai trailer|"cobra kai official trailer"...
998    Ozuna|"Reggaeton"|"Odisea"|"Odisea The Album"|...
999    Friends|"pop music"|"marshmello friends"|"anne...
Name: tags, Length: 1000, dtype: object>


In [45]:
#limpiamos la estructura de los datos

In [32]:
df_sample['ntags'] = df_sample['tags'].str.replace('[{}]'.format(string.punctuation), ' ', regex=True)
print(df_sample['ntags'].head())


0                                                none 
1    the four  singing   dj khaled   diddy   meghan...
2                                                none 
3    words you cant say anymore  the m word   now t...
4    SUPER JUNIOR  슈퍼주니어   은혁   동해   예성   신동   시원  ...
Name: ntags, dtype: object


In [33]:
#excluimos palabras que no agregan valor
tfidf=TfidfVectorizer(stop_words='english')
#rellenar espacios sin datos con ''
df_sample['ntags'] = df_sample['ntags'].fillna('')


In [46]:

tfidf_matrix = tfidf.fit_transform(df_sample['ntags'])

print(len(tfidf.vocabulary_))


6725


In [35]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [36]:
#borrar duplicados
indices= pd.Series(df_sample.index, index=df_sample['title']).drop_duplicates()

In [41]:
# Verificar si el título está en el índice y manejar duplicados si es necesario
def obtener_recomendacion(title, cosine_sim=cosine_sim):
    if title not in indices:
        print(f"El título '{title}' no está en el índice.")
        return []

    idx = indices[title]

    # En caso de que idx sea una serie con múltiples índices, tomar el primero
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # ordenar por similitud
    sim_scores = sim_scores[1:5]  # obtener los 4 más similares (excluyendo el propio)
    movie_indices = [i[0] for i in sim_scores]

    return df_sample['title'].iloc[movie_indices]


In [45]:
titulo=str(df_sample.iloc[2]['title'])
print('tu seleccionaste:' +titulo+' ,tu recomendacion es: ')
print(obtener_recomendacion(titulo))

tu seleccionaste:The Best of Keith Jackson ,tu recomendacion es: 
1    Cheyenne Elliott Performs Wings | Season 1 Ep....
2                            The Best of Keith Jackson
3                          The Return of Whiteface Man
4    SUPER JUNIOR 슈퍼주니어 'Lo Siento (Feat. Leslie Gr...
Name: title, dtype: object
